In [1]:
# encoding moel can run on either the betas or the raw timeseries (convolved with a design matrix)
# the easiest way will be to run on the betas, in which case it's just a linear model

# plan:
# 1. mask image using graymatter (or ROI)
# 2. Run linear model for each feature
# 3. Turn mask back into brain.

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
import os
os.chdir('..')

In [4]:
import json

import numpy as np
from tqdm import tqdm
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, cross_val_score, permutation_test_score

from joblib import Parallel, delayed
import multiprocessing

MAX_CPU = multiprocessing.cpu_count() // 2

In [5]:
from analysis.fmri.analogy_rsa import \
downsample_rdms_df_by_factor, create_models, models_to_df, plotmodels,\
roi_rdm, run_rsa_dfs, subject_rdms, plot_results

from analysis.fmri.analogy_utils import \
    projectSettings, analysisSettings, contrastSettings, order, \
    pu, pa, pv, compile_models, rsa, save_rois, load_rois, load_betas

paths = projectSettings["filepaths"]["hoffPaths"]

/u/home/n/njchiang/.conda/envs/fmri/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


Loading JSON config from config/analyses.json
Loading JSON config from config/contrasts.json
Loading label file from: labels/trialorder_rsa_absorted.csv


In [6]:
accuracies = pu.load_labels(paths["code"], "labels", "group_accuracy.csv").set_index("Trial")

raw_models_df = pu.load_labels(os.path.join(paths["code"], "labels", "raw_models.csv"))

bart270_df = (raw_models_df
            .reset_index(drop=True)
            .set_index("ABTag")[
                [c for c in raw_models_df.columns if "rstpostprob270" in c]
            ])

bartpower_df = (raw_models_df
            .reset_index(drop=True)
            .set_index("ABTag")[
                [c for c in raw_models_df.columns if "bart79power" in c]
            ])  

bartnorm_df = (raw_models_df
            .reset_index(drop=True)
            .set_index("ABTag")[
                [c for c in raw_models_df.columns if "bart79norm" in c]
            ])  

bart_df = (raw_models_df
            .reset_index(drop=True)
            .set_index("ABTag")[
                [c for c in raw_models_df.columns if "rstpostprob79" in c]
            ])  

w2vc_df = (raw_models_df
            .reset_index(drop=True)
            .set_index("ABTag")[
                [c for c in raw_models_df.columns if "concatword" in c]
            ])  

w2vd_df = (raw_models_df
            .reset_index(drop=True)
            .set_index("ABTag")[
                [c for c in raw_models_df.columns if "w2vdiff" in c]
            ])  

model_names = ["Word2vec-diff", "Word2vec-concat", "BART"]
bartnorm_df.shape, bart270_df.shape

Loading label file from: /u/project/monti/Analysis/Analogy/code/analogy-fmri/labels/group_accuracy.csv
Loading label file from: /u/project/monti/Analysis/Analogy/code/analogy-fmri/labels/raw_models.csv


((288, 79), (288, 270))

# Write AB Trials

In [19]:
## Write out file for hongjing
maskname = "graymatter-bin_mask"
# maskname = "dfc-mod-left-ba10_mask"

for sub in projectSettings["subjects"]:
    
# sub = "sub-01"
    mask = pu.load_img(paths["root"], "derivatives", sub, "masks", "{}.nii.gz".format(maskname))
    fmri_data, labels, bg_image = load_betas(projectSettings, 
                                             sub, 
                                             t="cope-LSS", center=True, scale=False)

    fmri_data = pu.mask_img(fmri_data, mask)

    trials = labels[(labels.AB == 1)]

    fmri_data = fmri_data[trials.index]
    labels = labels.loc[trials.index]
    write = {"fmri_data": fmri_data, "wordpair": trials.ABTag.values}
    for name, model in zip(["bartpower", "bartnorm", "bart"], [bartpower_df, bartnorm_df, bart_df]):
        write[name] = model.loc[[tag for tag in labels["ABTag"]], :].values
    pu.save_mat_data(os.path.join(paths["root"], "derivatives", sub, "mat_export", "{}_betas.mat".format(sub)), **write)

Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/masks/graymatter-bin_mask.nii.gz
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/func/sub-01_task-analogy_run-01_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/func/sub-01_task-analogy_run-02_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/func/sub-01_task-analogy_run-03_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/func/sub-01_task-analogy_run-04_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/func/sub-01_task-analogy_run-05_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/func/sub-01_task-analogy_run-06_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analog

Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-06/func/sub-06_task-analogy_run-08_events.csv
Concatenating 8 images
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-06/reg/BOLD_template.nii.gz
Applying mask
Saving mat data to /u/project/monti/Analysis/Analogy/derivatives/sub-06/mat_export/sub-06_betas.mat
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-07/masks/graymatter-bin_mask.nii.gz
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-07/func/sub-07_task-analogy_run-01_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-07/func/sub-07_task-analogy_run-02_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-07/func/sub-07_task-analogy_run-03_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-07/func/sub-07_task-analogy_run-04_eve

Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-12/func/sub-12_task-analogy_run-05_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-12/func/sub-12_task-analogy_run-06_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-12/func/sub-12_task-analogy_run-07_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-12/func/sub-12_task-analogy_run-08_events.csv
Concatenating 8 images
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-12/reg/BOLD_template.nii.gz
Applying mask
Saving mat data to /u/project/monti/Analysis/Analogy/derivatives/sub-12/mat_export/sub-12_betas.mat
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-13/masks/graymatter-bin_mask.nii.gz
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-13/func/sub-13_task-analogy_run-01_eve

# Write CD Trials

In [29]:
## Write out file for hongjing
raw_models_df = pu.load_labels(os.path.join(paths["code"], "labels", "raw_models.csv"))
bart_df = (raw_models_df
            .reset_index(drop=True)
            .set_index("ABTag")[
                [c for c in raw_models_df.columns if "rstpostprob79" in c]]).iloc[::2]


maskname = "graymatter-bin_mask"
# maskname = "dfc-mod-left-ba10_mask"

for sub in projectSettings["subjects"]:

# sub = "sub-01"
    mask = pu.load_img(paths["root"], "derivatives", sub, "masks", "{}.nii.gz".format(maskname))
    fmri_data, labels, bg_image = load_betas(projectSettings, 
                                             sub, 
                                             t="cope-LSS", center=True, scale=False)

    fmri_data = pu.mask_img(fmri_data, mask)

    trials = labels[(labels.CD == 1)]
    trials["ABTag"] = trials["TrialTag"].apply(lambda x: x.split("::")[0])

    dists = np.concatenate(
        [rsa.pdist(bart_df.loc[
            [t.split("::")[0], t.split("::")[1]]], 
               metric="cosine") 
         for t in trials.TrialTag])
    
    fmri_data = fmri_data[trials.index]
    labels = labels.loc[trials.index]
    write = {"fmri_data": fmri_data, 
             "CD_wordpair": trials.CDTag.values, 
             "AB_wordpair": trials.ABTag.values, 
             "Match": trials.Match.values.astype(int), 
             "dists": dists}
#     for name, model in zip(["bartpower", "bartnorm", "bart"], [bartpower_df, bartnorm_df, bart_df]):
#         write[name] = model.loc[[tag for tag in labels["CDTag"]], :].values
    pu.save_mat_data(os.path.join(paths["root"], "derivatives", sub, "mat_export", "{}_CD_betas.mat".format(sub)), **write)

Loading label file from: /u/project/monti/Analysis/Analogy/code/analogy-fmri/labels/raw_models.csv
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/masks/graymatter-bin_mask.nii.gz
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/func/sub-01_task-analogy_run-01_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/func/sub-01_task-analogy_run-02_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/func/sub-01_task-analogy_run-03_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/func/sub-01_task-analogy_run-04_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/func/sub-01_task-analogy_run-05_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/func/sub-01_task-

/u/home/n/njchiang/.conda/envs/fmri/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Saving mat data to /u/project/monti/Analysis/Analogy/derivatives/sub-01/mat_export/sub-01_CD_betas.mat
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-02/masks/graymatter-bin_mask.nii.gz
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-02/func/sub-02_task-analogy_run-01_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-02/func/sub-02_task-analogy_run-02_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-02/func/sub-02_task-analogy_run-03_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-02/func/sub-02_task-analogy_run-04_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-02/func/sub-02_task-analogy_run-05_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-02/func/sub-02_t

Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-07/func/sub-07_task-analogy_run-07_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-07/func/sub-07_task-analogy_run-08_events.csv
Concatenating 8 images
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-07/reg/BOLD_template.nii.gz
Applying mask
Saving mat data to /u/project/monti/Analysis/Analogy/derivatives/sub-07/mat_export/sub-07_CD_betas.mat
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-08/masks/graymatter-bin_mask.nii.gz
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-08/func/sub-08_task-analogy_run-01_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-08/func/sub-08_task-analogy_run-02_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-08/func/sub-08_task-analogy_run-03_

Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-13/func/sub-13_task-analogy_run-04_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-13/func/sub-13_task-analogy_run-05_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-13/func/sub-13_task-analogy_run-06_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-13/func/sub-13_task-analogy_run-07_events.csv
centering image
Loading label file from: /u/project/monti/Analysis/Analogy/derivatives/sub-13/func/sub-13_task-analogy_run-08_events.csv
Concatenating 8 images
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-13/reg/BOLD_template.nii.gz
Applying mask
Saving mat data to /u/project/monti/Analysis/Analogy/derivatives/sub-13/mat_export/sub-13_CD_betas.mat
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-14/masks/graymatter-bin_m

# Write significance maps

In [6]:
%%bash
# register significance map to each subject

. ~/.bashrc

module load fsl
cd /u/project/monti/Analysis/Analogy

fslmaths analysis/group/multivariate/rsa/pearson/n1000-AB_BART79_tfce_corrp_tstat1 -thr .95 -bin derivatives/standard/masks/ab_bart79_95-bin_mask
# fslmaths analysis/group/multivariate/rsa/pearson/n1000-AB_BART79_tfce_corrp_tstat1 -thr .95 derivatives/standard/masks/ab_bart79_95_mask
cp analysis/group/multivariate/rsa/pearson/n1000-AB_BART79_tfce_corrp_tstat1.nii.gz derivatives/standard/masks/ab_bart79_pval.nii.gz

# echo ${m}
for s in 01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16
do
    echo ${s}
    for m in BARTnorm BARTpower BART Word2vec-diff Word2vec-concat
    do
        cp analysis/sub-${s}/encoding/sub-${s}_${m}_cope-LSS_AB_cv-relation.nii.gz \
            derivatives/sub-${s}/masks/ab_${m}_corr.nii.gz
    done
    
    flirt -in derivatives/standard/masks/ab_bart79_95-bin_mask \
        -out derivatives/sub-${s}/masks/ab_bart79_95-bin_mask \
        -ref derivatives/sub-${s}/reg/BOLD_template \
        -applyxfm -init derivatives/sub-${s}/reg/standard_to_BOLD_template.mat
    flirt -in derivatives/standard/masks/ab_bart79_pval \
        -out derivatives/sub-${s}/masks/ab_bart79_pval \
        -ref derivatives/sub-${s}/reg/BOLD_template \
        -applyxfm -init derivatives/sub-${s}/reg/standard_to_BOLD_template.mat
    fslmaths derivatives/sub-${s}/masks/ab_bart79_95-bin_mask -bin derivatives/sub-${s}/masks/ab_bart79_95-bin_mask
done

01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16


In [7]:
analysisSettings["roi"]["masks"]["bart_sig_thr"] = 'ab_bart79_95-bin_mask'
analysisSettings["roi"]["masks"]["bart_pval"] = 'ab_bart79_pval'
analysisSettings["roi"]["masks"]["bart_corr"] = 'ab_BART_corr'
analysisSettings["roi"]["masks"]["bartnorm_corr"] = 'ab_BARTnorm_corr'
analysisSettings["roi"]["masks"]["bartpower_corr"] = 'ab_BARTpower_corr'
analysisSettings["roi"]["masks"]["w2vc_corr"] = 'ab_Word2vec-concat_corr'
analysisSettings["roi"]["masks"]["w2vd_corr"] = 'ab_Word2vec-diff_corr'


In [8]:
## Write out gramatter idx for hongjing
maskname = "graymatter-bin_mask"
for sub in projectSettings["subjects"]:
    mask = pu.load_img(paths["root"], "derivatives", sub, "masks", "{}.nii.gz".format(maskname))
    write = {}
    for rname, rfile in analysisSettings["roi"]["masks"].items():
        fmri_data = pu.load_img(paths["root"], "derivatives", sub, "masks", "{}.nii.gz".format(rfile))
        fmri_data = pu.mask_img(fmri_data, mask)
        print("{}- {}".format(rname, fmri_data.sum()))
        write[rname.replace("-", "_")] = fmri_data
    pu.save_mat_data(os.path.join(paths["root"], "derivatives", sub, "mat_export", "{}_idx.mat".format(sub)), **write)

Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/masks/graymatter-bin_mask.nii.gz
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/masks/dfc-left-ba10_mask.nii.gz
Applying mask
BA10m- 1049.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/masks/neubert-left-FPl-dil2-handmasked_mask.nii.gz
Applying mask
BA10l- 709.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/masks/harvardoxford-thr25-left-ba44_mask.nii.gz
Applying mask
BA44- 702.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/masks/dfc-left-ba9_mask.nii.gz
Applying mask
BA9- 569.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/masks/harvardoxford-thr25-left-ba45_mask.nii.gz
Applying mask
BA45- 692.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-01/masks/dfc-mod-left-ba46_mask.nii.gz
Applying mask
BA46- 893.0
Reading file from: /u/project/monti/Analysis/Analogy/deriva

dlpfc-power- 128.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-02/masks/anat-left-vlpfc-power2011.nii.gz
Applying mask
vlpfc-power- 102.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-02/masks/anat-left-spl-power2011.nii.gz
Applying mask
spl-power- 142.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-02/masks/anat-left-ipl-power2011.nii.gz
Applying mask
ipl-power- 105.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-02/masks/mars-left-SPLd_mask.nii.gz
Applying mask
spl_d- 722.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-02/masks/mars-left-SPLe_mask.nii.gz
Applying mask
spl_e- 498.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-02/masks/mars-left-IPLd_mask.nii.gz
Applying mask
ipl_d- 285.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-02/masks/mars-left-IPLe_mask.nii.gz
Applying mask
ipl_e- 285.0
Reading file from: /u/project

BA45- 722.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-04/masks/dfc-mod-left-ba46_mask.nii.gz
Applying mask
BA46- 953.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-04/masks/neubert-mod-left-ba47_mask.nii.gz
Applying mask
BA47- 975.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-04/masks/harvardoxford-thr25-left-pSTG_mask.nii.gz
Applying mask
lpstg- 1049.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-04/masks/harvardoxford-thr25-left-aITG_mask.nii.gz
Applying mask
laitg- 446.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-04/masks/harvardoxford-thr25-left-pMTG_mask.nii.gz
Applying mask
lpmtg- 860.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-04/masks/harvardoxford-thr25-left-temporalpole_mask.nii.gz
Applying mask
ltp- 2314.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-04/masks/harvardoxford-thr25-left-toMTG_mask.nii.gz

ipl_d- 305.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-05/masks/mars-left-IPLe_mask.nii.gz
Applying mask
ipl_e- 305.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-05/masks/csf-bin_mask.nii.gz
Applying mask
csf- 159.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-05/masks/ab_bart79_95-bin_mask.nii.gz
Applying mask
bart_sig_thr- 2501.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-05/masks/ab_bart79_pval.nii.gz
Applying mask
bart_pval- 11286.842788027572
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-05/masks/ab_BART_corr.nii.gz
Applying mask
bart_corr- -5021.236794701422
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-05/masks/ab_BARTnorm_corr.nii.gz
Applying mask
bartnorm_corr- -2269.05115821112
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-05/masks/ab_BARTpower_corr.nii.gz
Applying mask
bartpower_corr- -4610.845154544666
Rea

ltp- 2146.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-07/masks/harvardoxford-thr25-left-toMTG_mask.nii.gz
Applying mask
ltomtg- 898.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-07/masks/harvardoxford-thr25-left-aSTG_mask.nii.gz
Applying mask
lastg- 290.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-07/masks/juelich-left-SPL-7A_mask.nii.gz
Applying mask
lspl- 1865.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-07/masks/juelich-left-IPS_mask.nii.gz
Applying mask
lips- 1030.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-07/masks/juelich-left-IPL-PFm_mask.nii.gz
Applying mask
psmg- 682.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-07/masks/juelich-left-IPL-Pga_mask.nii.gz
Applying mask
lag- 959.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-07/masks/aal-v1_mask.nii.gz
Applying mask
v1- 3974.0
Reading file from: /u/proj

...

lag- 1317.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-10/masks/aal-v1_mask.nii.gz
Applying mask
v1- 5208.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-10/masks/anat-left-rlpfc-hobeika2016.nii.gz
Applying mask
rlpfc-hobeika- 160.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-10/masks/anat-left-rlpfc-power2011.nii.gz
Applying mask
rlpfc-power- 165.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-10/masks/anat-left-dlpfc-power2011.nii.gz
Applying mask
dlpfc-power- 155.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-10/masks/anat-left-vlpfc-power2011.nii.gz
Applying mask
vlpfc-power- 135.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-10/masks/anat-left-spl-power2011.nii.gz
Applying mask
spl-power- 160.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-10/masks/anat-left-ipl-power2011.nii.gz
Applying mask
ipl-power- 134.0
Readin

BA10m- 1462.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-12/masks/neubert-left-FPl-dil2-handmasked_mask.nii.gz
Applying mask
BA10l- 968.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-12/masks/harvardoxford-thr25-left-ba44_mask.nii.gz
Applying mask
BA44- 948.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-12/masks/dfc-left-ba9_mask.nii.gz
Applying mask
BA9- 746.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-12/masks/harvardoxford-thr25-left-ba45_mask.nii.gz
Applying mask
BA45- 917.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-12/masks/dfc-mod-left-ba46_mask.nii.gz
Applying mask
BA46- 1223.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-12/masks/neubert-mod-left-ba47_mask.nii.gz
Applying mask
BA47- 1238.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-12/masks/harvardoxford-thr25-left-pSTG_mask.nii.gz
Applying mask
lpstg- 1

spl-power- 115.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-13/masks/anat-left-ipl-power2011.nii.gz
Applying mask
ipl-power- 139.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-13/masks/mars-left-SPLd_mask.nii.gz
Applying mask
spl_d- 700.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-13/masks/mars-left-SPLe_mask.nii.gz
Applying mask
spl_e- 494.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-13/masks/mars-left-IPLd_mask.nii.gz
Applying mask
ipl_d- 330.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-13/masks/mars-left-IPLe_mask.nii.gz
Applying mask
ipl_e- 330.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-13/masks/csf-bin_mask.nii.gz
Applying mask
csf- 163.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-13/masks/ab_bart79_95-bin_mask.nii.gz
Applying mask
bart_sig_thr- 2554.0
Reading file from: /u/project/monti/Analysis/Analo

BA47- 957.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-15/masks/harvardoxford-thr25-left-pSTG_mask.nii.gz
Applying mask
lpstg- 1108.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-15/masks/harvardoxford-thr25-left-aITG_mask.nii.gz
Applying mask
laitg- 419.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-15/masks/harvardoxford-thr25-left-pMTG_mask.nii.gz
Applying mask
lpmtg- 831.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-15/masks/harvardoxford-thr25-left-temporalpole_mask.nii.gz
Applying mask
ltp- 2344.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-15/masks/harvardoxford-thr25-left-toMTG_mask.nii.gz
Applying mask
ltomtg- 1008.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-15/masks/harvardoxford-thr25-left-aSTG_mask.nii.gz
Applying mask
lastg- 319.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-15/masks/juelich-left-SPL-

csf- 157.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-16/masks/ab_bart79_95-bin_mask.nii.gz
Applying mask
bart_sig_thr- 2524.0
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-16/masks/ab_bart79_pval.nii.gz
Applying mask
bart_pval- 11038.153340982102
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-16/masks/ab_BART_corr.nii.gz
Applying mask
bart_corr- -4166.96300327091
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-16/masks/ab_BARTnorm_corr.nii.gz
Applying mask
bartnorm_corr- -3725.60549656491
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-16/masks/ab_BARTpower_corr.nii.gz
Applying mask
bartpower_corr- -6857.194024115813
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-16/masks/ab_Word2vec-concat_corr.nii.gz
Applying mask
w2vc_corr- -3371.4827214105844
Reading file from: /u/project/monti/Analysis/Analogy/derivatives/sub-16/masks/ab_Word2vec-diff_corr.nii.gz
